# White Matter dataset

-> to script with param

- slides
- masks
- downsample
- size
- output
- area

In [15]:
from brainseg.slide_provider import get_mask_from_slidepath
from brainseg.utils import save_data
from brainseg.streamlit.manager import init_curation_dataset, fill_curation_dataset
from brainseg.slide_provider import SlideHandler
from brainseg.provider import provider
from brainseg.loader import Loader

import aicspylibczi
import os
import glob
from itertools import product
from pathlib import Path
from PIL import Image
import numpy as np
import re

In [19]:
slides = Path("/media/nathan/LaCie/REGISTRATION_PROJECT/M148_RH/")
masks = Path("/media/nathan/LaCie/REGISTRATION_PROJECT/FILLED_MASK_M148_RH/")

In [25]:
def slides_with_mask(slides_path, masks_path):
    keep = []
    for slidepath in os.listdir(slides):
        if get_mask_from_slidepath(slidepath, masks) is not None:
            keep.append(slidepath)
    return keep

In [21]:
def parse_curation_name(name):
    try:
        return re.findall("(.+)_\d+_\d+", name)[0]
    except:
        return None

In [35]:
def only_new_slides(slides_path, cur_dts_path):
    current_slides = set(filter(lambda x: x is not None,
                                map(parse_curation_name,
                                    os.listdir(cur_dts_path))))
    slides = list(set(slides_path) - current_slides)
    return slides

In [36]:
keep = slides_with_mask(slides, masks)

In [37]:
len(keep)

35

In [38]:
keep = only_new_slides(keep, "/media/nathan/LaCie/Data/whitematter_curated2")

In [39]:
len(keep)

30

In [5]:
def build_czi_patches(fp, downsample, size):
    # non overlapping patches
    slide = aicspylibczi.CziFile(str(fp))
    bbox = slide.get_mosaic_bounding_box()
    max_size = int(bbox.h / downsample), int(bbox.w / downsample)
    patches = [
        dict(
            slidepath=str(fp),
            ori_x=i,
            ori_y=j,
            downscale=downsample,
            size=size
        )
        for i, j in product(
            range(0, bbox.w, size * downsample),
            range(0, bbox.h, size * downsample)
        )
    ]
    
    return patches

In [6]:
all_patches = sum([build_czi_patches(slides / sl, 32, 224) for sl in keep], [])
all_patches = list(map(lambda x: ("whitematter", x), all_patches))

In [7]:
# save_data(all_patches, "data/dataset/tinyset.pck")

In [8]:
sh = SlideHandler("/media/nathan/LaCie/M148_RH/", "/media/nathan/KINGSTON/BrainMapping/filled_masks_148/")
provider.register(sh)

In [9]:
def preprocess_curation_dataset(d, x, y):
    x = x.astype(np.uint8)
    y = y * 255

    data_name = f"{Path(d[1]['slidepath']).name}_{d[1]['ori_x']}_{d[1]['ori_y']}"
    res = dict(
        data_name=data_name,
        image=Image.fromarray(x),
        mask=Image.fromarray(y)
    )
    return res

In [10]:
loader = Loader(all_patches, preprocess=preprocess_curation_dataset)

In [11]:
root_cur_dts = "/media/nathan/LaCie/Data/whitematter_curated2"
init_curation_dataset(root_cur_dts)

In [12]:
fill_curation_dataset(root_cur_dts, loader, name="first")

In [2]:
import imgaug.augmenters as iaa

seq = iaa.Sequential([
    iaa.Fliplr(0.5), # horizontal flips
    iaa.Flipud(0.5), # horizontal flips
    iaa.Rot90([0, 1, 2, 3]),
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-25, 25),
        shear=(-8, 8)
    )
]) 

seq(image=image, segmentation_maps=segmap)